In [1]:
import pandas as pd
import re
pd.set_option('display.max_rows', None)

In [3]:
with open("semiotic_10000.txt") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [4]:
with open("semiotic_10_20000.txt") as f:
    content20 = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content20 = [x.strip() for x in content20] 

In [5]:
with open("semiotic_20_30000.txt") as f:
    content30 = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content30 = [x.strip() for x in content30] 

In [6]:
with open("semiotic_30_40000.txt") as f:
    content40 = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content40 = [x.strip() for x in content40] 

In [7]:
with open("semiotic_rest.txt") as f:
    contentrest = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
contentrest = [x.strip() for x in contentrest] 

In [8]:
with open("semiotic_sample.txt") as f:
    contentsample = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
contentsample = [x.strip() for x in contentsample] 

In [9]:
content_all = content + content20 + content30 + content40 + contentrest + contentsample

In [10]:
len(content_all)

40006

In [11]:
semiotic_classes = ['PUNCT', 'CARDINAL', 'ORDINAL', 'LETTERS', 
                    'DATE', 'TIME', 'MEASURE', 'SYMB', 'ABBR', 
                    'WLINK', 'DECIMAL', 'SPORT', 'RNUM', 
                    'FRACTION', 'DIGIT', 'TELEPHONE', 'DEL', 'MONEY']

In [12]:
def handle_sentence(sent, nr):
    # non-standard words are pulled out of the sentences
    sent1 = re.split('\}\s?|\s?\{', sent)
    # the whitespaces are deleted
    sent2 = list(filter(None, sent1))
    # put all into lists
    sent3 = [word.split(", ") for word in sent2]
    # split the plain words and strip of whitespaces
    sent4 = [subsent[0].split() if len(subsent) == 1 else subsent for subsent in sent3]
    sent5 = [[word.strip() for word in sentence] for sentence in sent4]
    
    # append a PLAIN class to the plain words
    for sent in sent5:
        if sent[-1] not in semiotic_classes:
            sent.append('PLAIN')
            
    sent6 = []
    # Add the PLAIN class to each standard word
    for sent in sent5:
        if sent[-1] == 'PLAIN':
            for word in sent[:-1]:
                sublist = []
                sublist.extend([word, word, 'PLAIN'])
                sent6.append(sublist)
        else:
            sent6.append(sent)
    
    # insert sentence and word numbers
    for i in range(len(sent6)):
        sent6[i].insert(0, i)
        sent6[i].insert(0, nr)
            
    return sent6

# setja setninganúmer sem args hér


In [13]:
def make_dataframe(data):
    all_data = []
    for i in range(len(data)):
        tmp = handle_sentence(data[i], i+1)
        all_data.append(tmp)
    flat_data = [item for sublist in all_data for item in sublist]
    df = pd.DataFrame(flat_data, columns = ['sentence_id', 'token_id', 'before', 'after', 'semiotic'])
    df = df[['sentence_id', 'token_id', 'semiotic', 'before', 'after']]
    return df

In [15]:
content_df_all = make_dataframe(content_all)

In [16]:
sentences_with_sport = list(content_df_all[content_df_all['semiotic'] == 'SPORT']['sentence_id'].unique())

In [17]:
content_sport = content_df_all[content_df_all['sentence_id'].isin(sentences_with_sport)]

In [18]:
content_other = content_df_all[~content_df_all['sentence_id'].isin(sentences_with_sport)]

In [19]:
content_sport.to_csv("dataframe_sport_0503.csv")

In [20]:
content_other.to_csv("dataframe_other_0503.csv")

In [249]:
content_df_all.to_csv("dataframe_all_0802.csv")